In [1]:
import pandas as pd
import numpy as np
from utils import get_species, get_labels, get_labels_all

In [2]:
X, y, y_all = get_species(), get_labels(), get_labels_all()

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score, accuracy_score

In [4]:
np.unique(y_all)

array(['ACVD', 'Adenoma', 'Adenoma (MP)', 'Advanced Dementia',
       'Behcet’s disease', 'Breast Cancer', 'CD', 'CRC', 'Cancer',
       "Crohn''s disease", 'Crohns disease',
       'End-stage renal disease (ESRD)', 'Graves’ disease', 'Healthy',
       'Hypertension (HTN)', 'IGT', 'Large adenoma', 'Liver Cirrhosis',
       'NAFLD', 'Non–small cell lung cancer (NSCLC)', 'Obese', 'Obesity',
       'Overweight', 'Pancreatic cancer', 'Renal cell carcinoma (RCC)',
       'Rheumatoid Arthritis', 'Rheumatoid arthritis', 'Small adenoma',
       'T2D', 'Ulcerative colitis', 'Underweight', 'adenoma',
       'advanced adenoma', 'ankylosing spondylitis', 'carcinoma',
       'schizophrenia', 'ulcerative colitis'], dtype=object)

In [5]:
X = X[list(filter(lambda x : "virus" not in x and "unclassified" not in x, X.columns))]

In [6]:
phenotype_list = list(np.unique(y_all))
old_phenotype_list = list(np.unique(y_all))
phenotype_list

['ACVD',
 'Adenoma',
 'Adenoma (MP)',
 'Advanced Dementia',
 'Behcet’s disease',
 'Breast Cancer',
 'CD',
 'CRC',
 'Cancer',
 "Crohn''s disease",
 'Crohns disease',
 'End-stage renal disease (ESRD)',
 'Graves’ disease',
 'Healthy',
 'Hypertension (HTN)',
 'IGT',
 'Large adenoma',
 'Liver Cirrhosis',
 'NAFLD',
 'Non–small cell lung cancer (NSCLC)',
 'Obese',
 'Obesity',
 'Overweight',
 'Pancreatic cancer',
 'Renal cell carcinoma (RCC)',
 'Rheumatoid Arthritis',
 'Rheumatoid arthritis',
 'Small adenoma',
 'T2D',
 'Ulcerative colitis',
 'Underweight',
 'adenoma',
 'advanced adenoma',
 'ankylosing spondylitis',
 'carcinoma',
 'schizophrenia',
 'ulcerative colitis']

In [14]:
from sklearn.preprocessing import OrdinalEncoder



In [11]:
from sklearn.model_selection import LeaveOneGroupOut
logo = LeaveOneGroupOut()

In [15]:
from tqdm import tqdm
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression

removed_list = []

while phenotype_list != []:
    best_phenotype_list = []
    best_score = -1
    best_removed = None
    for i in tqdm(range(len(phenotype_list))):
        removed = phenotype_list[i]
        if removed == "Healthy":
            continue
        curr_phenotype_list = phenotype_list[:i] + phenotype_list[i + 1:]
        index = y_all.isin(curr_phenotype_list)
        X_curr = X.iloc[index.values, :]
        y_curr = y.iloc[index.values, :]
        
        c = 0.00001
        
        sample_studies = np.array(X_curr.index.get_level_values(0))
        o = OrdinalEncoder()
        groups = o.fit_transform(sample_studies.reshape((len(sample_studies), 1))).flatten()
        
        clf = LogisticRegression(random_state=42, penalty="l1", solver="liblinear", C=1, class_weight="balanced")
        predictions = cross_val_predict(clf, X_curr > c, y_curr.values.flatten(), 
                         groups=groups, cv=logo, verbose=0, n_jobs=-1)
        
        score = balanced_accuracy_score(y_curr, predictions)
        
        if score > best_score:
            best_score = score
            best_phenotype_list = curr_phenotype_list
            best_removed = removed
    removed_list.append(best_removed)
    phenotype_list = best_phenotype_list
    print("best score:", best_score)
    print("Removed phenotypes:", removed_list)

  0%|                                                                        | 0/37 [00:00<?, ?it/s][Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:   22.4s finished
  3%|█▋                                                              | 1/37 [00:22<13:34, 22.63s/it][Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:   15.1s finished
  5%|███▍                                                            | 2/37 [00:37<10:41, 18.33s/it][Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:   16.3s finished
  8%|█████▏                                                          |

[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:   17.7s finished
 76%|███████████████████████████████████████████████▋               | 28/37 [08:35<02:50, 18.90s/it][Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done  68 out of  68 | elapsed:   22.3s finished
 78%|█████████████████████████████████████████████████▍             | 29/37 [08:57<02:40, 20.00s/it][Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:   19.7s finished
 81%|███████████████████████████████████████████████████            | 30/37 [09:17<02:19, 19.98s/it][Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    3.7s
[Paralle

best score: 0.6148444921729792
Removed phenotypes: ['Overweight']


  0%|                                                                        | 0/36 [00:00<?, ?it/s][Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    8.2s
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:   36.5s finished
  3%|█▊                                                              | 1/36 [00:36<21:24, 36.71s/it][Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:   20.1s finished
  6%|███▌                                                            | 2/36 [00:57<15:21, 27.09s/it][Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:   28.2s finished
  8%|█████▎                                                          |

[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done  68 out of  68 | elapsed:   19.1s finished
 78%|████████████████████████████████████████████████▏             | 28/36 [50:20<25:19, 189.93s/it][Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:   16.7s finished
 81%|█████████████████████████████████████████████████▉            | 29/36 [50:37<16:06, 138.13s/it][Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:   22.8s finished
 83%|███████████████████████████████████████████████████▋          | 30/36 [51:00<10:21, 103.63s/it][Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    5.3s
[Paralle

...

  0%|                                                                        | 0/29 [00:00<?, ?it/s][Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done  67 out of  67 | elapsed:   17.6s finished
  3%|██▏                                                             | 1/29 [00:17<08:18, 17.79s/it][Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done  67 out of  67 | elapsed:   10.4s finished
  7%|████▍                                                           | 2/29 [00:28<06:05, 13.55s/it][Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done  67 out of  67 | elapsed:  4.9min finished
 10%|██████▎                                                      | 3/

[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done  67 out of  67 | elapsed:   12.5s finished
 97%|████████████████████████████████████████████████████████████▊  | 28/29 [10:44<00:12, 12.87s/it][Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done  67 out of  67 | elapsed:   10.8s finished
100%|███████████████████████████████████████████████████████████████| 29/29 [10:55<00:00, 22.61s/it]


best score: 0.6939756132933925
Removed phenotypes: ['Overweight', 'Obesity', 'Underweight', 'IGT', 'schizophrenia', 'Hypertension (HTN)', 'Advanced Dementia', 'Ulcerative colitis', 'Adenoma']


  0%|                                                                        | 0/28 [00:00<?, ?it/s][Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done  67 out of  67 | elapsed:   10.9s finished
  4%|██▎                                                             | 1/28 [00:11<05:00, 11.14s/it][Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done  67 out of  67 | elapsed:   10.6s finished
  7%|████▌                                                           | 2/28 [00:21<04:44, 10.94s/it][Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done  67 out of  67 | elapsed:   10.2s finished
 11%|██████▊                                                         |

KeyboardInterrupt: 